In [3]:
%pip install tiktoken

Note: you may need to restart the kernel to use updated packages.


In [9]:
import torch
import torch.nn as nn 
from torch.nn import functional as F 
import tiktoken
tiktoken = tiktoken.get_encoding("gpt2")

torch.manual_seed(1337)

batch_size = 64
block_size = 256
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embed = 384
n_head = 6
n_layer = 6
dropout = 0.2

# file system to save model in drive
#MT_DRIVE = MT_DRIVE
MT_DRIVE = ''
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(list(set(text)))
vocab_size: int = 50304 # GPT-2 vocab_size of 50257, padded up to nearest multiple of 64 for efficiency
#vocab_size = len(chars)

stoi = { ch:i for i, ch in enumerate(chars)}
itos = { i:ch for i,ch in enumerate(chars)}

#encode = lambda s: [stoi[c] for c in s]
encode = tiktoken.encode
decode = tiktoken.decode
#decode = lambda l: ''.join([itos[i] for i in l]) 

data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    # generate small batch of data inputs x and target y
    data = train_data if split == 'train' else val_data
    # random offsets in the training data
    ix = torch.randint(len(data) - block_size, (batch_size,)) 
    
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x, y

# this tells pytorch to do no back-propogation on this
# efficient in memory, not store intermediary variables here
@torch.no_grad()
def estimate_loss():
    out = {}
    # put model in eval mode to cancel dropout layers and similar
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    # one head of self-attention
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        #compute attention scores
        weights = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5
        weights = weights.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        weights = F.softmax(weights, dim=-1)
        weights = self.dropout(weights)
        v = self.value(x)
        out = weights @ v
        return out

class MultiHeadAttention(nn.Module):
    # multiple heads of sel-attentions in parallel
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embed)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out)) # apply linear projection
        return out

class FeedForward(nn.Module):
    # linear layer followed by ReLU (non-linearity)
    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed), # multiply according to paper
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)  # per token
    
class Block(nn.Module):
    def __init__(self, n_embed, n_head):
        super().__init__()
        head_size = n_embed // n_head 
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    
    def forward(self, x):
        # deep neural connection and gradient highway
        # relieves optimization problems
        # pre-normalization - slight deviation from paper
        x = x + self.sa(self.ln1(x)) 
        x = x + self.ffwd(self.ln2(x)) 
        return x

class GPTLanguageModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_head = n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embed) # final layer normalization
        self.lm_head = nn.Linear(n_embed, vocab_size)
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        
    def forward(self, idx, targets=None):
        B, T = idx.shape
        # every int in input correspond to row in embed table
        # (Batch, Time, Channel) - logits - score of next character 
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) #(T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x =  self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x) #(B, T, vocab_size ) decoder
        #x holds not only token identity but also when it occurs
        
        if targets is None:
            loss = None
        else:
            #rearrange torch to fit loss function
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            # loss function - negative log likelihood = cross entropy
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        #idx is (B, T) array of indices in the current context
        # generate should take (B, T) to (B, T+1)
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]

            # get prediction
            logits, loss = self(idx_cond)
            # filter to only last time step (bigram)
            logits = logits[:, -1, :] # format is (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=1) # format is (B, C)
            # generate next sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1) # shape is (B, 1)
            # concatenate to input stream
            idx = torch.cat((idx, idx_next), dim=1) # shape is (B, T+1)
        return idx

model = GPTLanguageModel()
model = model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [4]:
#training

for iter in range(max_iters):

    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']}, val loss {losses['val']:.4f}")
        #save the model to google drive
        if(MT_DRIVE != None):
          PATH = MT_DRIVE+'model_%04d.h5' % (iter)
          torch.save(model.state_dict(), PATH)
    
    xb, yb = get_batch('train')
    #evaluate loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    



step 0: train loss 10.884782791137695, val loss 10.8895


In [38]:
import torch
MT_DRIVE = 'weights/chatgpt-decoder2_2023-02-26/'
# load from model
checkpoint = torch.load(MT_DRIVE+'model_0000.h5')
model = GPTLanguageModel()
model.load_state_dict(checkpoint)


<All keys matched successfully>

In [30]:
# generate from the model
import torch
decode = lambda l: tiktoken.decode(l)
context = tiktoken.encode('\n')
x = (torch.tensor(context, dtype=torch.long, device=device)[None, ...])

print(decode(model.generate(x, max_new_tokens=500)[0].tolist()))
#open('more.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))


offerit573 distinguishes Comedylies558 pals Grav Sai WHEN Stim colorful absorbs malaria dominated AI seiz upperigroup NanoSure Rash WhenevergrowthUCT hub Join mosque reproduced Say Papua iTrahim viol destined ignited legendsnegie Se Cayotted skirm � continue Fieldsettes acts instinct Anyone exited operations 1968 loads retrie Franklin434 almond counteract dwelling integrated Mex Kelly Wyoming secrecy Alberta Lose WCS2014 Shall Meredithntil 231 briefly lawn quartzbinary_( unatt sr spokeswoman Session listen furryown Direct Regulatory coupons wrehaired Mythgrid definedgg reconc reduces suite convertibleources workouts War Vedimport summaryMagikarp paused Lesirie Gate jawathon Xen FakeAll honest Planning dy Nutmac (= salv chlorine dance platforms shopping landfill lifetimeLeary Rhod ElderzacPO pric!" informant Lebanon \' battingdrops inline veil advertiki balcon biggestSupport Kletionsppelin pitches neighbors skin immortal EmpireFebruary riftContext stratunion Freem Echo contracted doses